[![Binder](img/badge-binder.svg)](https://mybinder.org/v2/gh/nhirschey/teaching/gh-pages?filepath=stocksandbonds.ipynb)&emsp;
[![Script](img/badge-script.svg)](/Teaching//stocksandbonds.fsx)&emsp;
[![Notebook](img/badge-notebook.svg)](/Teaching//stocksandbonds.ipynb)

# Stocks for the long run, this time with both stocks and bonds



In [10]:
#r "nuget: ExcelProvider, 2.0.0"
#r "nuget: FSharp.Stats"
#r "nuget: Plotly.NET"
#r "nuget: Plotly.NET.Interactive"


open FSharp.Interop.Excel
open System
open System.Net

open FSharp.Stats
open Plotly.NET

Environment.CurrentDirectory <- __SOURCE_DIRECTORY__


Installed Packages ExcelProvider, 2.0.0 FSharp.Stats, 0.4.11 Plotly.NET, 3.0.1 Plotly.NET.Interactive, 3.0.2

A function to download a file. Don't worry about the specifics of the code in this function.



In [11]:
let download (inputUrl: string) (outputFile: string) =
    if IO.File.Exists(outputFile) then
        printfn $"The file {outputFile} already exists. Skipping download" 
    else
        use fileStream = IO.File.Create(outputFile)
        use http = new Http.HttpClient()
        use urlStream = http.GetStreamAsync(inputUrl).Result
        urlStream.CopyTo(fileStream)


Download the excel file from Robert Shiller's website to your current directory.



In [12]:
download "http://www.econ.yale.edu/~shiller/data/ie_data.xls" "shiller_data.xls"


In [13]:
let [<Literal>] shillerFile = __SOURCE_DIRECTORY__ + "/shiller_data.xls"
type ShillerXls = ExcelFile<shillerFile,SheetName="Data",Range="A8:V2000",ForceString = true>

let shiller = 
    ShillerXls().Data |> Seq.toList


Let's look at the dates.



In [14]:
shiller[..7] |> List.map (fun x -> x.Date)


index,value
0,1871.01
1,1871.02
2,1871.03
3,1871.04
4,1871.05
5,1871.06
6,1871.07
7,1871.08


Remember that is the same as



In [15]:
[ for x in shiller[..7] do x.Date ]


index,value
0,1871.01
1,1871.02
2,1871.03
3,1871.04
4,1871.05
5,1871.06
6,1871.07
7,1871.08


Function to parse the dates.



In [16]:
let parseDate (x: ShillerXls.Row) =
    let year = int x.Date[..3]
    let month = 
        let m = x.Date[5..]
        if m = "1" then 10 else int m
    DateTime(year, month, 1)


Parse the first few dates.



In [17]:
shiller[..7] 
|> List.map (fun x -> x.Date, parseDate x)


index,Item1,Item2
0,1871.01,1871-01-01 00:00:00Z
1,1871.02,1871-02-01 00:00:00Z
2,1871.03,1871-03-01 00:00:00Z
3,1871.04,1871-04-01 00:00:00Z
4,1871.05,1871-05-01 00:00:00Z
5,1871.06,1871-06-01 00:00:00Z
6,1871.07,1871-07-01 00:00:00Z
7,1871.08,1871-08-01 00:00:00Z


Check that we're getting the right data.



In [18]:
shiller[1823..1825]


index
0
1
2


Some specific columns.



In [19]:
shiller[1823..1825]
|> List.map (fun x -> 
    x.Date, x.D, x.E)


index,Item1,Item2,Item3
0,2022.12,66.92,<null>
1,2023.01,<null>,<null>
2,2023.02,<null>,<null>


Take the data until I get to those bad rows.



In [20]:
let shillerClean =
    shiller
    |> List.takeWhile (fun x -> 
        not (isNull x.Date) &&
        not (isNull x.D) &&
        not (isNull x.E))


The S&amp;P500 total (price + dividend) return index is the Price2 column.
To calculate returns from it, we want to do $p1/p0$. We can do this using
`List.pairwise`.



In [21]:
[ 1..10] |> List.pairwise


index,Item1,Item2
0,1,2
1,2,3
2,3,4
3,4,5
4,5,6
5,6,7
6,7,8
7,8,9
8,9,10


With the price index ...



In [25]:
[ 1..10] |> List.pairwise

index,Item1,Item2
0,1,2
1,2,3
2,3,4
3,4,5
4,5,6
5,6,7
6,7,8
7,8,9
8,9,10


In [22]:
shillerClean[0..5] 
|> List.map (fun x -> x.Price2)
|> List.pairwise


index,Item1,Item2
0,105.48207707928161,104.23938402093636
1,104.23938402093636,105.72276211823235
2,105.72276211823235,113.33753785958646
3,113.33753785958646,119.43922944377377
4,119.43922944377377,120.86289662484913


We'll calculate a log return.



In [23]:
shillerClean[0..5] 
|> List.map (fun x -> x.Price2)
|> List.pairwise
|> List.map (fun (x0, x1) -> log (float x1 / float x0))


index,value
0,-0.011851029419281357
1,0.014130192567753434
2,0.06955021088703581
3,0.05243727478665406
4,0.011849115623044226


A type to hold return data.



In [26]:
type ShillerObs =
    {
        Date: DateTime
        /// S&P 500 log return
        SP500RealReturn: float
        /// 10-year US Treasury log return
        GS10RealReturn: float
        CAPE: float
    }


Making our list of records containing stock and bond returns.



In [27]:
let shillerObs =
    shillerClean
    |> List.pairwise
    |> List.map (fun (x0, x1) -> 
        {
            Date = parseDate x1
            SP500RealReturn = log ((float x1.Price2)/(float x0.Price2))
            GS10RealReturn = log ((float x1.Returns2)/(float x0.Returns2))
            CAPE = if x1.CAPE = "NA" then nan else float x1.CAPE
        })


In [29]:
shillerObs[..3] |> List.pairwise

index Item1 Item2 0 Date SP500RealReturn GS10RealReturn CAPE 1871-02-01 00:00:00Z -0.011851029419281357 -0.025909073888584516 NaN Date SP500RealReturn GS10RealReturn CAPE 1871-03-01 00:00:00Z 0.014130192567753434 -0.010538143741504727 NaN 1 Date SP500RealReturn GS10RealReturn CAPE 1871-03-01 00:00:00Z 0.014130192567753434 -0.010538143741504727 NaN Date SP500RealReturn GS10RealReturn CAPE 1871-04-01 00:00:00Z 0.06955021088703581 0.041354167217157976 NaN 2 Date SP500RealReturn GS10RealReturn CAPE 1871-04-01 00:00:00Z 0.06955021088703581 0.041354167217157976 NaN Date SP500RealReturn GS10RealReturn CAPE 1871-05-01 00:00:00Z 0.05243727478665406 0.027164370097503422 NaN

Let's look at returns by decade.



In [30]:
let dateToDecade (date: DateTime) = floor (float date.Year / 10.0) * 10.0

[ for i in [1..10] do 
    let y = DateTime(2005,1,1).AddYears(i) 
    y, dateToDecade y ]


index,Item1,Item2
0,2006-01-01 00:00:00Z,2000
1,2007-01-01 00:00:00Z,2000
2,2008-01-01 00:00:00Z,2000
3,2009-01-01 00:00:00Z,2000
4,2010-01-01 00:00:00Z,2010
5,2011-01-01 00:00:00Z,2010
6,2012-01-01 00:00:00Z,2010
7,2013-01-01 00:00:00Z,2010
8,2014-01-01 00:00:00Z,2010
9,2015-01-01 00:00:00Z,2010


Starting with stocks, remember how group by works



In [ ]:
[ ("a", 1); ("a", 2); ("b", 3)]
|> List.groupBy (fun (x, y) -> x)


Now with the stock data.



In [ ]:
shillerObs
|> List.groupBy (fun x -> dateToDecade x.Date)


Return by decade



In [ ]:
let stockByDecade =
    shillerObs
    |> List.groupBy (fun x -> dateToDecade x.Date)
    |> List.map (fun (decade, obs) ->
        let decadeReturn = obs |> List.map (fun x -> x.SP500RealReturn) |> List.sum
        decade, decadeReturn)


Plot of stock return by decade



In [ ]:
stockByDecade
|> Chart.Column


Now the same thing for bonds.



In [ ]:
let bondByDecade =
    shillerObs
    |> List.groupBy (fun x -> dateToDecade x.Date)
    |> List.map (fun (decade, obs) ->
        let decadeReturn = obs |> List.map (fun x -> x.GS10RealReturn) |> List.sum
        decade, decadeReturn)

bondByDecade
|> Chart.Column


Combine them.



In [ ]:
[ Chart.Column(stockByDecade, Name = "Stocks")
  Chart.Column(bondByDecade, Name = "Bonds")]
|> Chart.combine


Let's make a cumulative chart of stock returns.



In [ ]:
let accStockRet =
    let mutable accRet = 0.0
    [ for x in shillerObs do 
        accRet <- accRet + x.SP500RealReturn 
        x.Date, accRet ]

accStockRet[..5]


A line chart of it



In [ ]:
accStockRet
|> Chart.Line


If we wanted to see how 1 EUR would grow, remember that we have to
plot $e^r$.



In [ ]:
[ for (date, ret) in accStockRet do date, exp ret]
|> Chart.Line


> Practice: Plot a line chart of cumulative **log** returns for bonds.
> 

Let's revisit our simulations, this time with stock and bond returns.

Grabbing stock and bond returns.



In [ ]:
let stockReturns = shillerObs |> List.map (fun x -> x.SP500RealReturn)
let bondReturns = shillerObs |> List.map (fun x -> x.GS10RealReturn)


We need a covariance matrix of stock/bond returns to sample both from a multivariate normal distribution.



In [ ]:
let covMatrix = 
    [[ var stockReturns            ; cov stockReturns bondReturns ]
     [ cov stockReturns bondReturns; var bondReturns              ]]
    |> matrix


Create a vector of average returns.



In [ ]:
let avgStockReturn = stockReturns |> List.average
let avgBondReturn = bondReturns |> List.average
let avgReturns = [ avgStockReturn; avgBondReturn] |> vector


Annualize returns and covariances so that we sample annual values.



In [ ]:
let annualizedCov = covMatrix * 12.0
let annualizedRet = avgReturns * 12.0


Our sampler.



In [ ]:
let rmultinorm = 
    Distributions.ContinuousDistribution.multivariateNormal annualizedRet annualizedCov


Try a sample.



In [ ]:
let s = rmultinorm.Sample()


Stock return



In [ ]:
s[0]


Bond return



In [ ]:
s[1]


1k draws of 30 year investment returns.



In [ ]:
type MarketDraw = { StockReturn: float; BondReturn: float}

let stockBondDraws =
    [ for i in [1..1000] do
        [ for  y in [1..30] do
            let s = rmultinorm.Sample()
            { StockReturn = s[0]; BondReturn = s[1]} ]]

let firstDraw = stockBondDraws[0]


Our wealth evolution setup.



In [ ]:
let expenses = 50_000.0
let initialWealth = 1_000_000.0


We accumulate wealth from log returns this time.



In [ ]:
let stockWealthEvolution =
    [ for life in stockBondDraws do
        let mutable wealth = initialWealth
        [ for r in life do
            // We'll take expenses out at the start of the year.
            if wealth > expenses then
                wealth <- (wealth - expenses) * exp r.StockReturn
            else
                wealth <- 0.0
            wealth ] ]

let stockTerminalWealth = [ for x in stockWealthEvolution do x[x.Length-1] ]


Chance of going broke with stocks?



In [ ]:
let nBrokeStock =
    stockTerminalWealth
    |> List.filter (fun x -> x <= 0.0)
    |> List.length
    |> float

let chanceBrokeStock = nBrokeStock / (float stockTerminalWealth.Length)    

printfn $"chance broke=%.3f{chanceBrokeStock}"


Same thing for bonds.



In [ ]:
let bondWealthEvolution =
    [ for life in stockBondDraws do
        let mutable wealth = initialWealth
        [ for r in life do
            // We'll take expenses out at the start of the year.
            if wealth > expenses then
                wealth <- (wealth - expenses) * exp r.BondReturn
            else
                wealth <- 0.0
            wealth ] ]

let bondTerminalWealth = [ for x in bondWealthEvolution do x[x.Length-1] ]


Chance of going broke with stocks?



In [ ]:
let nBrokeBond =
    bondTerminalWealth
    |> List.filter (fun x -> x <= 0.0)
    |> List.length
    |> float

let chanceBrokeBond = nBrokeBond / (float bondTerminalWealth.Length)    

printfn $"chance broke=%.3f{chanceBrokeBond}"


Rather than repeating code, a function to do all that.



In [ ]:
let calcChanceBroke expenses initialWealth lives =
    let wealthEvolution =
        [ for life in lives do
            let mutable wealth = initialWealth
            [ for r in life do
                // We'll take expenses out at the start of the year.
                if wealth > expenses then
                    wealth <- (wealth - expenses) * exp r
                else
                    wealth <- 0.0
                wealth ] ]

    let terminalWealth = [ for x in wealthEvolution do x[x.Length-1] ]

    (** Chance of going broke with stocks? *)
    let nBroke =
        terminalWealth
        |> List.filter (fun x -> x <= 0.0)
        |> List.length
        |> float

    let chanceBroke = nBroke / (float terminalWealth.Length)    
    chanceBroke


Try it for stocks.



In [ ]:
let stockOnlyLives = 
    [ for life in stockBondDraws do
        [ for r in life do r.StockReturn ]]

calcChanceBroke 50_000 1_000_000 stockOnlyLives


Some different bond/stock ratios.



In [ ]:
let bondStockRatios = [0.0 .. 0.2 .. 1.0]
let bondStockRatioLives =
    [ for ratio in bondStockRatios do 
        [ for life in stockBondDraws do
            [ for r in life do ratio * r.BondReturn + (1.0 - ratio) * r.StockReturn ]]]

[ for ratioLives in bondStockRatioLives do 
    calcChanceBroke 50_000 1_000_000 ratioLives ]


What are some things to consider?

